In [9]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import optree

nn.AdaptiveAvgPool2d

torch.nn.modules.pooling.AdaptiveAvgPool2d

In [21]:
def abc(b):
    def cdf(b):
        if b is None:
            b = 4
        print(b)
    cdf(b)
    print(b)
    
def pqr():
    yield 5
    return 3

a = pqr()
try:
    print(next(a))
    print(next(a))
except StopIteration as e:
    print(e)

5
3


In [16]:
n = torch.distributions.MultivariateNormal(torch.tensor([1.0]), torch.eye(1))

AttributeError: 'MultivariateNormal' object has no attribute 'kl_divergence'

In [21]:
transforms._presets.ImageClassification(crop_size=224)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.vit_b_16(weights=False).to(device)
# model.heads = nn.Identity()
# model(torch.randn(2, 3, 224, 224).to(device)).shape
# model

In [3]:
path = os.path.expanduser('~/.cache/torch/')
mnist_train = datasets.MNIST(root=path, train=True, download=True)
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: /home/patel_zeel/.cache/torch/
    Split: Train

In [4]:
mnist_train.data = mnist_train.data.repeat(2, 1, 1)
mnist_train

Dataset MNIST
    Number of datapoints: 120000
    Root location: /home/patel_zeel/.cache/torch/
    Split: Train

In [5]:
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  